In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter



from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

from transformers import BertTokenizer

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Скачаем данные

In [ ]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')

df.head()

Удалим лишние столбцы, адекватно назовем колонки и закодируем таргеты.

In [ ]:
df = df.dropna(how='any', axis=1)
df.columns = ['target', 'text']
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['target'])

df['target_encoded'] = le.transform(df['target'])
del df['target']
df.columns = ['text', 'target']
df.head()

# Сделаем первичный анализ данных

На что хочется обратить внимание в первую очередь:
- Точно ли все сообщения на английском языке? Я прочитал описание датасета, там есть сообщения написанные жителями Сингапура, а еще часть сообщений взята с какого-то испанского сайта
- Сколько объектов в датасете
- Есть ли дисбаланс классов
- Какие бывают длины сообщений

Попробуем понять на каком языке сообщения, для простоты посмотрим на множество используемых символов, которые не попали в английский алфавит, цифры и пунктуацию.

In [ ]:
from collections import Counter
import string

chars = Counter()
for i, row in df.iterrows():
    for char in row['text']:
        chars[char] += 1
print('Символ    Частота')
for i in chars:
    if not re.match(r'[a-z]|[A-Z]|[0-9]|[{} ]'.format(string.punctuation), i):
        print(i, '        ', chars[i])

Да, символы не преднадлежащие английскому алфавиту есть, но их очень мало, и почти все отвечают названиям валют, так что будем считать что все тексты на английском.

Посчитаем сколько в датасете объектов.

In [ ]:
len(df)

Посмотрим, имеет ли место дисбаланс классов.

In [ ]:
target_count = df.groupby('target')['target'].agg('count').values
target_count

Нормальных текстов в 6 раз больше чем спама. Это дисбаланс классов, будем это учитывать при построении модели. Для этого будем использовать стратифицированный train-test split, а в качестве метрики качества дополнительно посчитаем ROC AUC.

Посмотрим, какие бывают длины сообщений.

In [ ]:
sms_lengths = df['text'].apply(lambda x: len(x.split(' ')))
ax = sms_lengths.plot.hist(bins=100, alpha=0.5, title='Распределение длин сообщений')
ax.set_xlabel('Длина сообщения')
ax.set_ylabel('Количество сообщений')

Почти все сообщения имеют длину до 50 слов, а самое большое 175. Одна из моделей которую хочется построить -  BERT, для нее можно будет привести все тексты к длине 50, меньшие тексты западить, большие обрезать.

# Сделаем базовую предобработку текстов

Немного почистим текст, удалим стоп-слова и сделаем стемминг.

In [ ]:
stop_words = stopwords.words('english')
stemmer = nltk.SnowballStemmer('english')

def preprocess_text(text):
    
    text = str(text).lower()
    text = re.sub('((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 'url', text)
    text = re.sub('[.,;-]', '', text)
    
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)

    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    
    return text

In [ ]:
df['text_clean'] = df['text'].apply(preprocess_text)
df.head()

# Попробуем простые модели

Я хочу сначала попробовать простые модели, которые не учитывают порядок слов в тексте. То есть сначала надо создать векторное представление отдельного текста, а потом передать его модели. Попробуем два векторых представления текста:
- TF-IDF
- Сумма GloVe эмбеддингов слов с весами TF-IDF

Преобразование корпуса текстов в tf-idf матрицу:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(corpus):
    
    vectorizer = TfidfVectorizer(analyzer='word',
                         ngram_range=(1, 2),
                         min_df=0,
                         max_df=0.5,
                         max_features=5000)
    
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    return tfidf_matrix

Скачаем предобученные GloVe эмбедденги.

In [ ]:
word2vec = dict()

with open('/kaggle/input/glovedata/glove.6B.200d.txt') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        word2vec[word] = vector_dimensions

Преобразование корпуса текстов в матрицу с суммами GloVe векторов текста с весами tf-idf:

In [ ]:
def doc2vec(corpus):
    
    vectorizer = TfidfVectorizer(analyzer='word',
                         ngram_range=(1, 2),
                         min_df=0,
                         max_df=0.5,
                         max_features=5000)
    
    tfidf_matrix = vectorizer.fit_transform(corpus)
    tfidf_feature_names = vectorizer.get_feature_names()
    
    d2v = []
    mask = [name in word2vec for name in tfidf_feature_names]
    mask = np.array(mask)
    tfidf_matrix_2 = tfidf_matrix[:, mask]
    vs = []
    for i in tfidf_feature_names:
        if i in word2vec:
            vs.append(word2vec[i])
    vs = np.array(vs)
    d2v = tfidf_matrix_2.dot(vs)
    return d2v

Вывод confusion_matrix:

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

def conf_matrix(y_true, y_pred):
    columns = ['HAM', 'SPAM']
    confm = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(confm, index=columns, columns=columns, dtype=np.int32)

    ax = sn.heatmap(df_cm, cmap='Blues', annot=True, fmt="d")

### Единый пайплайн для простых моделей

Для всех простых моделей сделаем единый пайплайн, в котором будет сразу все:
- Выбор способа векторизовать текст: через tf-idf, либо через GloVe с весами tf-idf
- Отслеживание accuracy на Train и Test
- Перебор параметров модели на CV решетке
- Вывод confusion matrix
- Вывод сэмплов на которых модель ошибается

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import plot_confusion_matrix

def full_pipeline(model, params, doc2vec_vectorizer=False, print_conf_matrix=False, print_false_classifications=False):
    
    if doc2vec_vectorizer:
        X = doc2vec(df['text_clean'])
    else:
        X = tf_idf(df['text_clean'])
    y = df['target'].to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=239)

    clf = GridSearchCV(model, params)
    clf.fit(X_train, y_train)

    print(f'Best classifier params: {clf.best_params_}')
    print(f'TRAIN accuracy: {round(clf.best_score_,3)}, TRAIN ROC AUC: \
    {round(roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1]), 3)}')
    
    y_pred = clf.predict(X_test)
    print(f'TEST accuracy: {round(accuracy_score(y_test, y_pred),3)}, TEST ROC AUC: \
    {round(roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]), 3)}')
    
    if print_conf_matrix:
        conf_matrix(y_test, y_pred)
    
    if print_false_classifications:
        classes = ['ham', 'spam']
        false_classification0 = []
        false_classification1 = []
        y_pred = clf.predict(X)
        for i in range(len(y)):
            if y[i] != y_pred[i]:
                if y[i] == 0:
                    false_classification0.append({'text': df.iloc[i]['text'], 'y_true': classes[y[i]], \
                                                  'y_pred': classes[y_pred[i]]})
                else:
                    false_classification1.append({'text': df.iloc[i]['text'], 'y_true': classes[y[i]], \
                                                  'y_pred': classes[y_pred[i]]})
        random.seed(241)
        miscl0 = random.sample(false_classification0, min(5, len(false_classification0)))
        miscl1 = random.sample(false_classification1, min(5, len(false_classification1)))
    
        print('\nExamples, when the model missclassify HAM:\n')
        for i in miscl0:
            print(f"Text: {i['text']}\nY_true: {i['y_true']}, Y_pred: {i['y_pred']}\n")
    
        print('\nExamples, when the model missclassify SPAM:\n')
        for i in miscl1:
            print(f"Text: {i['text']}\nY_true: {i['y_true']}, Y_pred: {i['y_pred']}\n")
        

### Обучаем и анализируем простые модели

Прогоним несколько простых моделей через этот пайплайн, посмотрим, что работает лучше всего.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


models = [LogisticRegression(), SVC(), MultinomialNB(), RandomForestClassifier(), KNeighborsClassifier()]
params = [{'penalty':['l2'], 'C':[0.1, 1, 10, 20], 'solver': ['liblinear']},\
          {'C':[0.5, 1, 2], 'kernel':['linear', 'rbf'], 'probability': [True]},\
          {},\
          {'max_depth':[5, 7, 12, 20], 'n_estimators':[20, 50, 100]},\
          {'n_neighbors':[2, 3, 5, 7, 10, 20]}]

print()
for model, param in zip(models, params):
    
    print('Model:', str(model), '\n')
    print('Training on tf-idf')
    full_pipeline(model, param)
    
    if str(model) == 'MultinomialNB()':
        print('\n')
        continue
    print('\nTraining on GloVe with tf-idf weights')
    full_pipeline(model, param, doc2vec_vectorizer=True)
    print('\n')

Мы видим что лучше всего себя показала SVM на tf-idf. Попробуем определить на основе выходов этой модели критерии, по которым она определяет является ли сообщение спамом или нет.


Посмотрим для этого на confusion_matrix и сэмплы на которых модель ошибается. Сэмплы возьмем не только из тестовой выборки, но и из трэйна, чтобы примеров ошибок обоих классов было больше.

In [ ]:
full_pipeline(SVC(), {'C':[2], 'kernel':['linear'], 'probability': [True]}, print_conf_matrix=True, print_false_classifications=True)

Можно сделать 2 простых наблюдения:
- Модель не очень заточена под детекцию цифр и чисел, хотя они очень свойственны для спама
- Модель склонна слишком часто относить в спам сообщения где много встречаются ключевые слова типа new, msg, хотя и обычные сообщения порой их содержат

Благодаря этим наблюдениям мы понимаем как еще можно улучшить модель, при желании.

Двинемся дальше. Мы посмотрели на работу простых моделей, которые никак не учитывают порядок слов в тексте. Теперь попробуем более сложную модель, которая уже учитывает порядок. Я попробую BERT.

# BERT

Для начала реализуем перевод сырых текстов в запаженные/обрезанные последовательности id токенов фиксированной длины, которые можно будет подать на вход в BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

def bert_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

Создадим входные данные для обучения модели.

In [ ]:
X = df['text_clean']
y = df['target'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=239)

target_text_len = 70

train_input_ids, train_attention_masks = bert_encode(X_train,target_text_len)

Прикрутим к BERT-у 2 линейных слоя с дропаутом и сигмоиду на выходе, чтобы модель выдавала нам вероятность 1-ого класса

In [ ]:
def create_model(bert_model):
    
    input_ids = tf.keras.Input(shape=(target_text_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(target_text_len,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

Загрузим предобученный BERT

In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Создадим и обучим модель

In [ ]:
model = create_model(bert_model)

history = model.fit(
    [train_input_ids, train_attention_masks],
    y_train,
    validation_split=0.2, 
    epochs=4,
    batch_size=10
)

Визуализируем процесс обучения модели.

In [ ]:
def plot_learning_curves(history, arr):
    fig, ax = plt.subplots(1, 2, figsize=(20, 5))
    for idx in range(2):
        ax[idx].plot(history.history[arr[idx][0]])
        ax[idx].plot(history.history[arr[idx][1]])
        ax[idx].legend([arr[idx][0], arr[idx][1]],fontsize=18)
        ax[idx].set_xlabel('A ',fontsize=16)
        ax[idx].set_ylabel('B',fontsize=16)
        ax[idx].set_title(arr[idx][0] + ' X ' + arr[idx][1],fontsize=16)

plot_learning_curves(history, [['loss', 'val_loss'],['accuracy', 'val_accuracy']])

Посчитаем ответы на тестовой выборке, посчитаем accuracy, ROC AUC на тестовой выборке.
Выведем сэмплы на которых модель ошибается (по всему датасету).
Вывыдем confusion_matrix.

In [ ]:
test_input_ids, test_attention_masks = bert_encode(X_test,target_text_len)

y_pred_proba = model.predict([test_input_ids, test_attention_masks])
y_pred_proba = y_pred_proba.reshape((-1))
y_pred_proba

y_pred = np.zeros_like(y_pred_proba)

for i in range(len(y_pred)):
    y_pred[i] = 1 if y_pred_proba[i] > 0.5 else 0

print(f'\nTEST accuracy: {round(accuracy_score(y_test, y_pred),3)}, TEST ROC AUC: \
{round(roc_auc_score(y_test, y_pred_proba), 3)}\n')

conf_matrix(y_test, y_pred)

# eval on full dataset
test_input_ids, test_attention_masks = bert_encode(X,target_text_len)

y_pred_proba = model.predict([test_input_ids, test_attention_masks])
y_pred_proba = y_pred_proba.reshape((-1))
y_pred_proba

y_pred = np.zeros_like(y_pred_proba, dtype=np.int32)

for i in range(len(y_pred)):
    y_pred[i] = 1 if y_pred_proba[i] > 0.5 else 0


classes = ['ham', 'spam']
false_classification0 = []
false_classification1 = []
for i in range(len(y)):
    if y[i] != y_pred[i]:
        if y[i] == 0:
            false_classification0.append({'text': df.iloc[i]['text'], 'y_true': classes[y[i]], \
                                          'y_pred': classes[y_pred[i]]})
        else:
            try:
                false_classification1.append({'text': df.iloc[i]['text'], 'y_true': classes[y[i]], \
                                              'y_pred': classes[y_pred[i]]})
            except Exception as e:
                print(df.iloc[i]['text'], y[i], y_pred[i])
                raise e

miscl0 = random.sample(false_classification0, min(5, len(false_classification0)))
miscl1 = random.sample(false_classification1, min(5, len(false_classification1)))

print('\nExamples, when the model missclassify HAM:\n')
for i in miscl0:
    print(f"Text: {i['text']}\nY_true: {i['y_true']}, Y_pred: {i['y_pred']}\n")

print('\nExamples, when the model missclassify SPAM:\n')
for i in miscl1:
    print(f"Text: {i['text']}\nY_true: {i['y_true']}, Y_pred: {i['y_pred']}\n")


Тут уже видно что примеры на которых модель ошибается не столь тривиальны даже для человека. Например, "Do you realize that in about 40 years, we'll have thousands of old ladies running around with tattoos?" - это отнесено к спаму, хотя понятно что это мог написать и твой знакомый. В общем модель ошибается на реально сложных по смыслу кейсах, которые находятся близко к границе разделения на классы. Особого пространтсва для улучшения модели уже нет.

# Выводы

Так как в сообщениях со спамом часто используются особые слова по типу url-адресов и телефонных номеров, простые модели, которые никак не учитывают порядок слов в тексте, работают довольно хорошо. Но лучший результат конечно же дает тяжелая модель, такая как BERT, которая учитывает порядок слов и лучше определяет контекст в сложных случаях.